In [1]:
import numpy as np
import _ccpm

from sqlalchemy import create_engine, inspect, ForeignKey, Column, Integer, Float, String, and_, null

from sqlalchemy.orm import relationship, remote, backref
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.orm.session import Session

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
class NetworkModel(Base):
    __tablename__ = 'models'

    id          = Column(Integer, primary_key = True, autoincrement=True)
    name        = Column(String, nullable=False)
    description = Column(String)
    
    #Work breakdown structure
    wbs        = relationship('WBSRecord', backref='model', cascade = 'all, delete-orphan')
    wbs_root   = relationship('WBSRecord', viewonly=True,
                              primaryjoin='and_(WBSRecord.model_id == NetworkModel.id, WBSRecord.parent_id == null())')
    
    # Network model elements
    activities = relationship('Activity' , backref='model', cascade = 'all, delete-orphan') # AoN AoA
    links      = relationship('Link'     , backref='model', cascade = 'all, delete-orphan') # AoN  -
    events     = relationship('Event'    , backref='model', cascade = 'all, delete-orphan') #  -  AoA
    
    # Visualization elements
    nodes      = relationship('Node'     , backref='model', cascade = 'all, delete-orphan')
    edges      = relationship('Edge'     , backref='model', cascade = 'all, delete-orphan')
    
    #----------------------------------------------------------------------------------------------
    def __init__(self, name, description='', session=None):
        self.name        = name
        self.description = description
        
        if session:
            assert isinstance(session, Session)

            session.add(self)
            session.flush() #We've got an id now

    #----------------------------------------------------------------------------------------------
    def describe(self):
        print(self.id, ':', self.name, ':', self.description)
    
    #----------------------------------------------------------------------------------------------
    def copy(self, name=None):
        if not name:
            cp = NetworkModel(description=self.description, 
                              name='Copy of: ' + self.name,
                              session=inspect(self).session)
        else:
            cp = NetworkModel(description=self.description, 
                              name=name,
                              session=inspect(self).session)
        for w in self.wbs_root:
            w.copy(model=cp)
        
        cp.links  = [l.copy(model=cp) for l in self.links]
        cp.events = [e.copy(model=cp) for e in self.events]
        
        cp.nodes  = [n.copy(model=cp) for n in self.nodes]
        cp.edges  = [e.copy(model=cp) for e in self.edges]

        inspect(self).session.flush()
        return cp
    
    #----------------------------------------------------------------------------------------------
    @property
    def _iwbs(self):
        return dict([(w.path, i) for i,w in enumerate(self.wbs)])
    
    def wbs_record(self, path):
        return self.wbs[self._iwbs[path]]
    
    #----------------------------------------------------------------------------------------------
    def compute_aoa(self):
        session = inspect(self).session
        session.flush()

        act_id = session.query(Activity.id).filter(Activity.model_id == self.id).all()
        act_id = np.array(tuple(zip(*act_id))[0])
        
        lnk_ids = tuple(zip(*session.query(Link.src_id, Link.dst_id).filter(Link.model_id == self.id).all()))
        lnk_src = np.array(lnk_ids[0])
        lnk_dst = np.array(lnk_ids[1])
        print(act_id, lnk_src, lnk_dst)
        
        print(_ccpm.compute_aoa(act_id, lnk_src, lnk_dst))
        #print(lnk_src)
        #print(lnk_dst)
        
        
    #----------------------------------------------------------------------------------------------
    def delete_aoa(self):
        if not self.events:
            return
        self.events = []
        self.nodes  = []
        self.edges  = []
        for a in self.activities:
            a.src = None
            a.dst = None
    #----------------------------------------------------------------------------------------------
    def refresh(self):
        sesion = inspect(self).session
        session.flush()
        session.expire(self)
    #----------------------------------------------------------------------------------------------
    def delete_activity(self, activity):
        if activity:
            assert isinstance(activity, Activity)
            assert activity.model_id == self.id

            self.activities.remove(activity)
            self.delete_aoa()
            self.refresh()
    
    #----------------------------------------------------------------------------------------------
    def delete_wbs_record(self, path):
        i = self._iwbs[path]
        
        if self.wbs[i].has_activities():
            self.delete_aoa()

        del self.wbs[i]
        self.refresh()

    #----------------------------------------------------------------------------------------------
    def delete_link(self, link):
        if link:
            assert isinstance(link, Link)
            assert link.model_id == self.id
            self.delete_aoa()
            self.links.remove(link)
            self.refresh()
            
    #----------------------------------------------------------------------------------------------
    #def __delete__(self, instance):
    #    print('Deleted model: ', self.id)
    #    super().__delete__(instance)
            

###################################################################################################
class WBSRecord(Base):
    __tablename__ = 'wbs'

    id        = Column(Integer, primary_key=True, autoincrement=True)
    model_id  = Column(Integer, ForeignKey('models.id'))
    parent_id = Column(Integer, ForeignKey('wbs.id'))
    
    path      = Column(String, nullable=False, index=True)
    name      = Column(String, nullable=False)
    
    children = relationship("WBSRecord",
                            remote_side=parent_id,
                            back_populates='parent',
                            cascade='all'
                           )
    
    parent  = relationship("WBSRecord",
                            remote_side=id,
                            back_populates='children',
                           )
    
    activity = relationship('Activity', 
                            backref='wbs', 
                            cascade = 'all', 
                            uselist=False)
    
    #----------------------------------------------------------------------------------------------
    def __init__(self, name, parent=None, model=None):
        if parent:
            assert isinstance(parent, WBSRecord)
            model = parent.model

        assert isinstance(model, NetworkModel)

        self.name   = name
        self.parent = parent
        self.model  = model
        self.path   = ''

        inspect(self).session.flush()
        # We've got an id now!

        if parent:
            self.path += parent.path + '.'
        self.path += str(self.id)

    #----------------------------------------------------------------------------------------------
    def _propagate_path(self, new_path_base):
        if self.parent:
            self.path = self.parent.path + '.' + str(self.id)
        else:
            self.path = str(self.id)

        for c in self.children:
            c._propagate_path(self.path)

    #----------------------------------------------------------------------------------------------
    def move_to(self, new_path_base):
        assert isinstance(new_path_base, str)
        
        if new_path_base != '':
            if new_path_base.startswith(self.path):
                raise ValueError('WTF R U doing motherfucker???')
            self.parent = self.model.comment(new_path_base)
        else:
            self.parent = None

        self._propagate_path(new_path_base)

        inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    def swap(self, wbs):
        assert isinstance(wbs, WBSRecord)
        
        assert wbs.activity
        assert isinstance(wbs.activity, Activity)
        
        assert self.activity
        assert isinstance(self.activity, Activity)
        
        temp = self.name
        self.name = wbs.name
        wbs.name  = temp
        
        temp = self.activity
        self.activity = wbs.activity
        wbs.activity  = temp
        
        inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    def copy(self, parent=None, model=None):
        cp = WBSRecord(self.name, parent=parent, model=model)

        if isinstance(self.activity, Activity):
            cp.activity = self.activity.copy(cp)

        cp.children = [c.copy(parent=cp) for c in self.children]

        return cp
    
    #----------------------------------------------------------------------------------------------
    def has_activities(self):
        if self.activity:
            assert isinstance(self.activity, Activity)
            return True
        
        for c in self.children:
            if c.has_activities():
                return True
        
        return False

    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'WBSRecord(model_id=%r id=%r, path=%r, name=%r)' % (
            self.model_id,
            self.id,
            self.path,
            self.name
        )

    #----------------------------------------------------------------------------------------------
    def dump(self, _level=0):
        return (
                '   ' * _level
                + repr(self)
                + "\n"
                + "".join([c.dump(_level + 1) for c in self.children])
        )

###################################################################################################
class Activity(Base):
    __tablename__ = 'activities'

    
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)
    id       = Column(Integer,                          primary_key=True)
    
    wbs_id   = Column(Integer, ForeignKey('wbs.id'))
        
    in_links  = relationship('Link', 
                             primaryjoin='and_(Activity.model_id == Link.model_id, Activity.id == Link.src_id)',
                             backref='src', cascade='all, delete-orphan')
    out_links = relationship('Link', 
                             primaryjoin='and_(Activity.model_id == Link.model_id, Activity.id == Link.dst_id)', 
                             backref='dst', cascade='all, delete-orphan')

    #Работы являются связями для событий
    src_id    = Column(Integer, ForeignKey('events.id'))
    dst_id    = Column(Integer, ForeignKey('events.id'))
    
    #CPM Data
    duration    = Column(Float, default=0.0) # Длительность работы
    early_start = Column(Float, default=0.0) # Ранний старт
    late_start  = Column(Float, default=0.0) # Поздний старт
    early_end   = Column(Float, default=0.0) # Ранний финиш
    late_end    = Column(Float, default=0.0) # Поздний финиш
    reserve     = Column(Float, default=0.0) # Резерв времени
    
    def __init__(self, wbs, id=None, src=None, dst=None, src_id=None, dst_id=None):
        assert isinstance(wbs, WBSRecord)

        if src:
            assert isinstance(src, Event)
            src_id = src.id

        if dst:
            assert isinstance(dst, Event)
            dst_id = dst.id

        self.model  = wbs.model
        #Обеспечили уникальность при генерации и возможность копирования
        self.id     = id if id else wbs.id
        self.wbs    = wbs
        self.src_id = src_id
        self.dst_id = dst_id
        
        if self.model:
            assert isinstance(self.model, NetworkModel)
            self.model.delete_aoa()
        
        inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    def __delete__(self, instance):
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('Will delete:', self)
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        super().__delete__(instance)
        
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Activity(model_id=%r id=%r, wbs.path=%r, wbs.name=%r, src_id=%r, dst_id=%r)' % (
            self.model_id,
            self.id,
            self.wbs.path,
            self.wbs.name,
            self.src_id,
            self.dst_id
        )
    
    #----------------------------------------------------------------------------------------------
    def copy(self, wbs):
        return Activity(wbs, src=self.src, dst=self.dst, id=self.id)

###################################################################################################
class Link(Base):
    __tablename__ = 'links'

    model_id  = Column(Integer, ForeignKey('models.id'    ),  primary_key=True)
    src_id    = Column(Integer, ForeignKey('activities.id'),  primary_key=True)
    dst_id    = Column(Integer, ForeignKey('activities.id'),  primary_key=True)
    
    def _clean_aoa(self):
        if self.model:
            assert isinstance(self.model, NetworkModel)
            self.model.delete_aoa()
            inspect(self).session.flush()
    
    def __init__(self, model=None, src=None, dst=None, src_id=None, dst_id=None):

        if model:
            assert isinstance(model, NetworkModel)
        
        if src:
            assert isinstance(src, Activity)
            src_id = src.id
            
        if dst:
            assert isinstance(dst, Activity)
            dst_id = dst.id
            
        if src and dst:
            assert src.model_id == dst.model_id

        if not model:
            model = src.model
                
        self.model    = model
        self.src_id   = src_id
        self.dst_id   = dst_id
        
        self._clean_aoa()

    #----------------------------------------------------------------------------------------------
    def set_src(self, node):
        assert isinstance(node, Node)
        assert node.model_id == self.model_id
        
        if node.id != self.src_id:
            self.src = node
            self._clean_aoa()
            inspect(self).session.flush()
            
    #----------------------------------------------------------------------------------------------
    def set_dst(self, node):
        assert isinstance(node, Node)
        assert node.model_id == self.model_id
        
        if node.id != self.dst_id:
            self.dst = node
            self._clean_aoa()
            inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    #def __delete__(self, instance):
    #    self._clean_aoa()
    #    super().__delete__(instance)
            
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Link(model_id=%r src_id=%r, dst_id=%r)' % (
            self.model_id,
            self.src_id,
            self.dst_id
        )
    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Link(src=self.src, dst=self.dst, model=model)

###################################################################################################
class Event(Base):
    __tablename__ = 'events'

    id       = Column(Integer,                          primary_key=True)
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)

    in_activities  = relationship('Activity', 
                             primaryjoin='and_(Event.model_id == Activity.model_id, Event.id == Activity.src_id)',
                             backref='src')
    out_activities = relationship('Activity', 
                             primaryjoin='and_(Event.model_id == Activity.model_id, Event.id == Activity.dst_id)', 
                             backref='dst')
    
    early   = Column(Float, default=0.0) #Ранее время наступления
    late    = Column(Float, default=0.0) #Позднее время наступления
    reserve = Column(Float, default=0.0) #Резерв времени
    
    def __init__(self, model, id):
        assert isinstance(model, NetworkModel)
        
        self.model = model
        self.id = id

    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Event(model_id=%r id=%r)' % (
            self.model_id,
            self.id
        )

    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Event(model=model,id=self.id)
    
    def __delete__(self, instance):
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('Will delete:', self)
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        super().__delete__(instance)
    
###################################################################################################
class Node(Base):
    __tablename__ = 'nodes'
    
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)
    id       = Column(Integer,                          primary_key=True)
    
    event_id = Column(Integer, ForeignKey('events.id'))
    
    x = Column(Float, default=0.0)
    y = Column(Float, default=0.0)
        
    in_edges  = relationship('Edge', 
                             primaryjoin='and_(Node.model_id == Edge.model_id, Node.id == Edge.src_id)',
                             backref='src', cascade = 'all, delete-orphan')
    out_edges = relationship('Edge', 
                             primaryjoin='and_(Node.model_id == Edge.model_id, Node.id == Edge.dst_id)', 
                             backref='dst', cascade = 'all, delete-orphan')
    
    event = relationship('Event',
                         primaryjoin='and_(Node.model_id == Event.model_id, Node.event_id == Event.id)')
    
    def __init__(self, id, model=None,  event=None):
        if model:
            assert isinstance(model, NetworkModel)
        
        if event:
            assert isinstance(event, Event)
            self.event_id = event.id

            if not model:
                model = event.model

        self.model = model
        self.id    = id

    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Node(model_id=%r, id=%r, event_id=%r)' % (
            self.model_id,
            self.id,
            self.event_id
        )
    
    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Node(self.id, model=model, event=self.event)

###################################################################################################
class Edge(Base):
    __tablename__ = 'edges'

    model_id  = Column(Integer, ForeignKey('models.id'),  primary_key=True)
    src_id    = Column(Integer, ForeignKey('nodes.id') ,  primary_key=True)
    dst_id    = Column(Integer, ForeignKey('nodes.id') ,  primary_key=True)
    
    def __init__(self, model=None, src=None, dst=None, src_id=None, dst_id=None):

        if model:
            assert isinstance(model, NetworkModel)
        
        if src:
            assert isinstance(src, Node)
            src_id = src.id
            
        if dst:
            assert isinstance(dst, Node)
            dst_id = dst.id
            
        if src and dst:
            assert src.model_id == dst.model_id

        if not model:
            model = src.model
                
        self.model  = model
        self.src_id = src_id
        self.dst_id = dst_id
            
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'Edge(model_id=%r src_id=%r, dst_id=%r)' % (
            self.model_id,
            self.src_id,
            self.dst_id
        )
    #----------------------------------------------------------------------------------------------
    def copy(self, model):
        return Edge(src=self.src, dst=self.dst, model=model)

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo = True)

In [4]:
from sqlalchemy.orm import sessionmaker
_Session = sessionmaker(bind=engine)
session = _Session()

In [5]:
Base.metadata.create_all(engine)

2021-09-05 14:14:03,957 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-05 14:14:03,958 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("models")
2021-09-05 14:14:03,958 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 14:14:03,959 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("models")
2021-09-05 14:14:03,960 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 14:14:03,961 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("wbs")
2021-09-05 14:14:03,962 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 14:14:03,963 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("wbs")
2021-09-05 14:14:03,963 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 14:14:03,964 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("activities")
2021-09-05 14:14:03,965 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-05 14:14:03,966 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("activities")
2021-09-05 14:14:03,967 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [6]:
nm = NetworkModel(name='First model!', session=session)

2021-09-05 14:14:04,090 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-05 14:14:04,092 INFO sqlalchemy.engine.Engine INSERT INTO models (name, description) VALUES (?, ?)
2021-09-05 14:14:04,093 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ('First model!', '')


In [7]:
nm.describe()

1 : First model! : 


In [8]:
w1 = WBSRecord('Test 1', model=nm)
a1 = Activity(w1)

2021-09-05 14:14:04,279 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-09-05 14:14:04,279 INFO sqlalchemy.engine.Engine [generated in 0.00087s] (1, None, '', 'Test 1')
2021-09-05 14:14:04,288 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities 
WHERE ? = activities.wbs_id
2021-09-05 14:14:04,288 INFO sqlalchemy.engine.Engine [generated in 0.00062s] (1,)
2021-09-05 14:14:04,291 INFO sqlalchemy.engine.Engine UPDATE wbs SET path=? WHERE wbs.id = ?
2021-09-05 14:14:04,29

In [9]:
w2 = WBSRecord('Test 2', model=nm)
w3 = WBSRecord('Test 3', parent=w1)
a3 = Activity(w3)

l1 = Link(src=a1, dst=a3)

ev1 = Event(model=nm, id=1)
ev2 = Event(model=nm, id=2)
ev3 = Event(model=nm, id=3)
print(session.query(Activity).all())

2021-09-05 14:14:04,412 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-09-05 14:14:04,413 INFO sqlalchemy.engine.Engine [cached since 0.1342s ago] (1, None, '', 'Test 2')
2021-09-05 14:14:04,415 INFO sqlalchemy.engine.Engine UPDATE wbs SET path=? WHERE wbs.id = ?
2021-09-05 14:14:04,415 INFO sqlalchemy.engine.Engine [cached since 0.1246s ago] ('2', 2)
2021-09-05 14:14:04,416 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-09-05 14:14:04,417 INFO sqlalchemy.engine.Engine [cached since 0.1386s ago] (1, 1, '', 'Test 3')
2021-09-05 14:14:04,419 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.

In [10]:
a1.src = ev1
a1.dst = ev2

a3.src = ev2
a3.dst = ev3
print(session.query(Activity).all())

2021-09-05 14:14:04,501 INFO sqlalchemy.engine.Engine UPDATE activities SET src_id=?, dst_id=? WHERE activities.model_id = ? AND activities.id = ?
2021-09-05 14:14:04,502 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ((1, 2, 1, 1), (2, 3, 1, 3))
2021-09-05 14:14:04,503 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities
2021-09-05 14:14:04,503 INFO sqlalchemy.engine.Engine [cached since 0.06741s ago] ()
[Activity(model_id=1 id=1, wbs.path='1', wbs.name='Test 1', src_id=1, dst_id=2), Activity(model_id=1 id=3, wbs.pat

In [11]:
n1 = Node(1, event=ev1)
n2 = Node(2, event=ev2)
n3 = Node(3, event=ev3)
n4 = Node(4, model=nm)
print(nm.nodes)

2021-09-05 14:14:04,622 INFO sqlalchemy.engine.Engine INSERT INTO nodes (model_id, id, event_id, x, y) VALUES (?, ?, ?, ?, ?)
2021-09-05 14:14:04,623 INFO sqlalchemy.engine.Engine [generated in 0.00073s] ((1, 1, 1, 0.0, 0.0), (1, 2, 2, 0.0, 0.0), (1, 3, 3, 0.0, 0.0), (1, 4, None, 0.0, 0.0))
2021-09-05 14:14:04,626 INFO sqlalchemy.engine.Engine SELECT nodes.model_id AS nodes_model_id, nodes.id AS nodes_id, nodes.event_id AS nodes_event_id, nodes.x AS nodes_x, nodes.y AS nodes_y 
FROM nodes 
WHERE ? = nodes.model_id
2021-09-05 14:14:04,626 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (1,)
[Node(model_id=1, id=1, event_id=1), Node(model_id=1, id=2, event_id=2), Node(model_id=1, id=3, event_id=3), Node(model_id=1, id=4, event_id=None)]


In [12]:
e1 = Edge(src=n1,dst=n2)
e2 = Edge(src=n2,dst=n4)
e3 = Edge(src=n4,dst=n3)
print(nm.edges)

2021-09-05 14:14:04,744 INFO sqlalchemy.engine.Engine INSERT INTO edges (model_id, src_id, dst_id) VALUES (?, ?, ?)
2021-09-05 14:14:04,745 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ((1, 1, 2), (1, 2, 4), (1, 4, 3))
2021-09-05 14:14:04,748 INFO sqlalchemy.engine.Engine SELECT edges.model_id AS edges_model_id, edges.src_id AS edges_src_id, edges.dst_id AS edges_dst_id 
FROM edges 
WHERE ? = edges.model_id
2021-09-05 14:14:04,748 INFO sqlalchemy.engine.Engine [generated in 0.00084s] (1,)
[Edge(model_id=1 src_id=1, dst_id=2), Edge(model_id=1 src_id=2, dst_id=4), Edge(model_id=1 src_id=4, dst_id=3)]


In [13]:
print(nm.wbs_root)

2021-09-05 14:14:04,855 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs 
WHERE wbs.model_id = ? AND wbs.parent_id IS NULL
2021-09-05 14:14:04,856 INFO sqlalchemy.engine.Engine [generated in 0.00059s] (1,)
[WBSRecord(model_id=1 id=1, path='1', name='Test 1'), WBSRecord(model_id=1 id=2, path='2', name='Test 2')]


In [14]:
print(nm.wbs)

2021-09-05 14:14:04,943 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs 
WHERE ? = wbs.model_id
2021-09-05 14:14:04,944 INFO sqlalchemy.engine.Engine [generated in 0.00102s] (1,)
[WBSRecord(model_id=1 id=1, path='1', name='Test 1'), WBSRecord(model_id=1 id=2, path='2', name='Test 2'), WBSRecord(model_id=1 id=3, path='1.3', name='Test 3')]


In [15]:
nm2 = nm.copy()

2021-09-05 14:14:05,372 INFO sqlalchemy.engine.Engine INSERT INTO models (name, description) VALUES (?, ?)
2021-09-05 14:14:05,373 INFO sqlalchemy.engine.Engine [cached since 1.281s ago] ('Copy of: First model!', '')
2021-09-05 14:14:05,375 INFO sqlalchemy.engine.Engine INSERT INTO wbs (model_id, parent_id, path, name) VALUES (?, ?, ?, ?)
2021-09-05 14:14:05,375 INFO sqlalchemy.engine.Engine [cached since 1.097s ago] (2, None, '', 'Test 1')
2021-09-05 14:14:05,378 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities 
WHERE ?

In [16]:
print(nm2.wbs)

2021-09-05 14:14:05,504 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs 
WHERE ? = wbs.model_id
2021-09-05 14:14:05,505 INFO sqlalchemy.engine.Engine [cached since 0.5619s ago] (2,)
[WBSRecord(model_id=2 id=4, path='4', name='Test 1'), WBSRecord(model_id=2 id=5, path='4.5', name='Test 3'), WBSRecord(model_id=2 id=6, path='6', name='Test 2')]


In [17]:
from sqlalchemy import select, text
print(session.query(WBSRecord).all())

2021-09-05 14:14:05,604 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs
2021-09-05 14:14:05,605 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ()
[WBSRecord(model_id=1 id=1, path='1', name='Test 1'), WBSRecord(model_id=1 id=2, path='2', name='Test 2'), WBSRecord(model_id=1 id=3, path='1.3', name='Test 3'), WBSRecord(model_id=2 id=4, path='4', name='Test 1'), WBSRecord(model_id=2 id=5, path='4.5', name='Test 3'), WBSRecord(model_id=2 id=6, path='6', name='Test 2')]


In [18]:
print(session.query(Activity).all())

2021-09-05 14:14:05,703 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities
2021-09-05 14:14:05,704 INFO sqlalchemy.engine.Engine [cached since 1.268s ago] ()
[Activity(model_id=1 id=1, wbs.path='1', wbs.name='Test 1', src_id=1, dst_id=2), Activity(model_id=1 id=3, wbs.path='1.3', wbs.name='Test 3', src_id=2, dst_id=3), Activity(model_id=2 id=1, wbs.path='4', wbs.name='Test 1', src_id=1, dst_id=2), Activity(model_id=2 id=3, wbs.path='4.5', wbs.name='Test 3', src_id=2, dst_id=3)]


In [19]:
print(session.query(Link).all())

2021-09-05 14:14:05,814 INFO sqlalchemy.engine.Engine SELECT links.model_id AS links_model_id, links.src_id AS links_src_id, links.dst_id AS links_dst_id 
FROM links
2021-09-05 14:14:05,814 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ()
[Link(model_id=1 src_id=1, dst_id=3), Link(model_id=2 src_id=1, dst_id=3)]


In [20]:
print(session.query(Event).all())

2021-09-05 14:14:05,904 INFO sqlalchemy.engine.Engine SELECT events.id AS events_id, events.model_id AS events_model_id, events.early AS events_early, events.late AS events_late, events.reserve AS events_reserve 
FROM events
2021-09-05 14:14:05,905 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ()
[Event(model_id=1 id=1), Event(model_id=1 id=2), Event(model_id=1 id=3), Event(model_id=2 id=1), Event(model_id=2 id=2), Event(model_id=2 id=3)]


In [21]:
print(nm2.nodes)

[Node(model_id=2, id=1, event_id=1), Node(model_id=2, id=2, event_id=2), Node(model_id=2, id=3, event_id=3), Node(model_id=2, id=4, event_id=None)]


In [22]:
print(nm2.edges)

[Edge(model_id=2 src_id=1, dst_id=2), Edge(model_id=2 src_id=2, dst_id=4), Edge(model_id=2 src_id=4, dst_id=3)]


In [23]:
nm3 = session.query(NetworkModel).get(2)

In [24]:
print(nm3.nodes)

[Node(model_id=2, id=1, event_id=1), Node(model_id=2, id=2, event_id=2), Node(model_id=2, id=3, event_id=3), Node(model_id=2, id=4, event_id=None)]


In [25]:
for m in session.query(NetworkModel).all():
    print(m.id)

2021-09-05 14:14:06,479 INFO sqlalchemy.engine.Engine SELECT models.id AS models_id, models.name AS models_name, models.description AS models_description 
FROM models
2021-09-05 14:14:06,479 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ()
1
2


In [26]:
print(inspect(nm3).session)
session.flush()

In [27]:
print(nm3.wbs)
print(nm3.activities)
print(nm3.links)
print(nm3.events)
print(nm3.nodes)
print(nm3.edges)

[WBSRecord(model_id=2 id=4, path='4', name='Test 1'), WBSRecord(model_id=2 id=5, path='4.5', name='Test 3'), WBSRecord(model_id=2 id=6, path='6', name='Test 2')]
2021-09-05 14:14:06,701 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities 
WHERE ? = activities.model_id
2021-09-05 14:14:06,701 INFO sqlalchemy.engine.Engine [generated in 0.00062s] (2,)
[Activity(model_id=2 id=1, wbs.path='4', wbs.name='Test 1', src_id=1, dst_id=2), Activity(model_id=2 id=3, wbs.path='4.5', wbs.name='Test 3', src_id=2, dst_id=3)]
[Link(model_id

In [28]:
nm3.delete_wbs_record('4')

2021-09-05 14:14:06,834 INFO sqlalchemy.engine.Engine SELECT links.model_id AS links_model_id, links.src_id AS links_src_id, links.dst_id AS links_dst_id 
FROM links 
WHERE ? = links.model_id AND ? = links.src_id
2021-09-05 14:14:06,835 INFO sqlalchemy.engine.Engine [generated in 0.00079s] (2, 3)
2021-09-05 14:14:06,838 INFO sqlalchemy.engine.Engine SELECT links.model_id AS links_model_id, links.src_id AS links_src_id, links.dst_id AS links_dst_id 
FROM links 
WHERE ? = links.model_id AND ? = links.dst_id
2021-09-05 14:14:06,838 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (2, 3)
2021-09-05 14:14:06,840 INFO sqlalchemy.engine.Engine SELECT links.model_id AS links_model_id, links.src_id AS links_src_id, links.dst_id AS links_dst_id 
FROM links 
WHERE ? = links.model_id AND ? = links.src_id
2021-09-05 14:14:06,841 INFO sqlalchemy.engine.Engine [cached since 0.006583s ago] (2, 1)
2021-09-05 14:14:06,842 INFO sqlalchemy.engine.Engine SELECT links.model_id AS links_model_id, links.

In [29]:
print(nm3.wbs)
print(nm3.activities)
print(nm3.links)
print(nm3.events)
print(nm3.nodes)
print(nm3.edges)

2021-09-05 14:14:06,921 INFO sqlalchemy.engine.Engine SELECT models.id AS models_id, models.name AS models_name, models.description AS models_description 
FROM models 
WHERE models.id = ?
2021-09-05 14:14:06,922 INFO sqlalchemy.engine.Engine [generated in 0.00091s] (2,)
2021-09-05 14:14:06,924 INFO sqlalchemy.engine.Engine SELECT wbs.id AS wbs_id, wbs.model_id AS wbs_model_id, wbs.parent_id AS wbs_parent_id, wbs.path AS wbs_path, wbs.name AS wbs_name 
FROM wbs 
WHERE ? = wbs.model_id
2021-09-05 14:14:06,924 INFO sqlalchemy.engine.Engine [cached since 1.982s ago] (2,)
[WBSRecord(model_id=2 id=6, path='6', name='Test 2')]
2021-09-05 14:14:06,926 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start

In [30]:
session.flush()
print(nm3.wbs)
print(nm3.activities)
print(nm3.links)
print(nm3.events)
print(nm3.nodes)
print(nm3.edges)

[WBSRecord(model_id=2 id=6, path='6', name='Test 2')]
[]
[]
[]
[]
[]


In [31]:
print(session.query(NetworkModel).all())

2021-09-05 14:14:07,129 INFO sqlalchemy.engine.Engine SELECT models.id AS models_id, models.name AS models_name, models.description AS models_description 
FROM models
2021-09-05 14:14:07,130 INFO sqlalchemy.engine.Engine [cached since 0.6512s ago] ()
[<__main__.NetworkModel object at 0x7ff8b7f5ee48>, <__main__.NetworkModel object at 0x7ff8b7d499e8>]


In [32]:
print(session.query(Activity).all())

2021-09-05 14:14:07,284 INFO sqlalchemy.engine.Engine SELECT activities.model_id AS activities_model_id, activities.id AS activities_id, activities.wbs_id AS activities_wbs_id, activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id, activities.duration AS activities_duration, activities.early_start AS activities_early_start, activities.late_start AS activities_late_start, activities.early_end AS activities_early_end, activities.late_end AS activities_late_end, activities.reserve AS activities_reserve 
FROM activities
2021-09-05 14:14:07,285 INFO sqlalchemy.engine.Engine [cached since 2.849s ago] ()
[Activity(model_id=1 id=1, wbs.path='1', wbs.name='Test 1', src_id=1, dst_id=2), Activity(model_id=1 id=3, wbs.path='1.3', wbs.name='Test 3', src_id=2, dst_id=3)]


In [33]:
nm.compute_aoa()

2021-09-05 14:14:07,417 INFO sqlalchemy.engine.Engine SELECT activities.id AS activities_id 
FROM activities 
WHERE activities.model_id = ?
2021-09-05 14:14:07,418 INFO sqlalchemy.engine.Engine [generated in 0.00060s] (1,)
2021-09-05 14:14:07,419 INFO sqlalchemy.engine.Engine SELECT links.src_id AS links_src_id, links.dst_id AS links_dst_id 
FROM links 
WHERE links.model_id = ?
2021-09-05 14:14:07,420 INFO sqlalchemy.engine.Engine [generated in 0.00054s] (1,)
[1 3] [1] [3]
(0, array([1, 2], dtype=uint16), array([2, 3], dtype=uint16), array([1]), array([3]))


In [34]:
nm.id

1

In [35]:
print(np.array(tuple(zip(*session.query(Activity.id).filter(Activity.model_id == nm.id).all()))[0]))

2021-09-05 14:14:07,661 INFO sqlalchemy.engine.Engine SELECT activities.id AS activities_id 
FROM activities 
WHERE activities.model_id = ?
2021-09-05 14:14:07,662 INFO sqlalchemy.engine.Engine [cached since 0.2448s ago] (1,)
[1 3]


In [36]:
print(list(zip(*((l.src_id, l.dst_id) for l in nm.links))))

[(1,), (3,)]


In [37]:
print(tuple(zip(*session.query(Activity.src_id, Activity.dst_id).filter(Activity.model_id == nm.id).all())))

2021-09-05 14:14:07,916 INFO sqlalchemy.engine.Engine SELECT activities.src_id AS activities_src_id, activities.dst_id AS activities_dst_id 
FROM activities 
WHERE activities.model_id = ?
2021-09-05 14:14:07,916 INFO sqlalchemy.engine.Engine [generated in 0.00060s] (1,)
((1, 2), (2, 3))
